In [ ]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
import json
import numpy as np
import tensorflow as tf
from scipy.spatial.distance import cosine
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

pb_model_path = "your_path"

def load_pb_model(pb_model_path):
    graph = tf.Graph()
    with graph.as_default():
        graph_def = tf.compat.v1.GraphDef()
        with tf.io.gfile.GFile(pb_model_path, "rb") as f:
            graph_def.ParseFromString(f.read())
            tf.import_graph_def(graph_def, name="")
    return graph

graph = load_pb_model(pb_model_path)
sess = tf.compat.v1.Session(graph=graph)

images_placeholder = graph.get_tensor_by_name("input:0")
embeddings_tensor = graph.get_tensor_by_name("embeddings:0")
phase_train_placeholder = graph.get_tensor_by_name("phase_train:0")

def preprocess_image(image):
    image = tf.image.resize(image, (160, 160))  
    image = tf.cast(image, dtype=tf.float32)  
    image = (image / 255.0) - 0.5  
    image = tf.expand_dims(image, axis=0)  
    return image


def generate_embedding_pb(image, sess, images_placeholder, embeddings_tensor, phase_train_placeholder):
    print("Shape of the input tensor:", image.shape)
    print("Type of the input tensor:", image.dtype)
    print("Check placeholder type:", images_placeholder.dtype)
    
    image_np = image.numpy()  
    feed_dict = {images_placeholder: image_np, phase_train_placeholder: False}
    
    try:
        embedding = sess.run(embeddings_tensor, feed_dict=feed_dict)
        return embedding[0]
    except Exception as e:
        print("Error during model execution:", str(e))
        return None



def load_images_and_labels(folder_path, sess, images_placeholder, embeddings_tensor, phase_train_placeholder):
    images = []
    labels = []
    processed_images_count = 0
    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith('.jpg'):
            processed_images_count += 1
            filepath = os.path.join(folder_path, filename)
            image = tf.io.read_file(filepath)
            image = tf.image.decode_jpeg(image)
            image = preprocess_image(image)
            try:
                label = int(filename.split('[')[1].split(']')[0])
                images.append(image)
                labels.append(label)
            except IndexError:
                print(f"Skipping file with unexpected filename format: {filename}")
                continue
    print(f"Processed images count: {processed_images_count}")
    embeddings = [generate_embedding_pb(img, sess, images_placeholder, embeddings_tensor, phase_train_placeholder) for img in images]
    return np.array(embeddings), np.array(labels)




image_folder = 'your_path'
embeddings, labels = load_images_and_labels(image_folder, sess, images_placeholder, embeddings_tensor, phase_train_placeholder)

if np.isnan(embeddings).any():
    print("Warning: NaN values found in embeddings.")
else:
    X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.2, random_state=42)
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)

results = {
    "accuracy": accuracy,
    "classification_report": report,
    "predictions": y_pred.tolist(),
    "true_labels": y_test.tolist()
}

print("Embeddings length:", len(embeddings))
print("Labels length:", len(labels))

print("Embeddings:", embeddings)
print("Labels:", labels)


if len(embeddings) != len(labels):
    print("Error: Embeddings and labels have different lengths.")


json_file_path = 'C:\\Users\\Acer Nitro 5\\Desktop\\STUDIA\\results.json'
with open(json_file_path, 'w') as f:
    json.dump(results, f, indent=4)

print(f"Results have been saved to {json_file_path}")


sess.close()

In [ ]:
import os
import json
import numpy as np
import tensorflow.compat.v1 as tf
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from scipy.spatial.distance import euclidean

tf.disable_v2_behavior()

pb_model_path = "your_path"

def load_pb_model(pb_model_path):
    graph = tf.Graph()
    with graph.as_default():
        graph_def = tf.compat.v1.GraphDef()
        with tf.io.gfile.GFile(pb_model_path, "rb") as f:
            graph_def.ParseFromString(f.read())
        tf.import_graph_def(graph_def, name="")
    return graph

def run_session():
    graph = load_pb_model(pb_model_path)
    with tf.Session(graph=graph) as sess:
        images_placeholder = graph.get_tensor_by_name("input:0")
        embeddings_tensor = graph.get_tensor_by_name("embeddings:0")
        phase_train_placeholder = graph.get_tensor_by_name("phase_train:0")

        def preprocess_and_generate_embedding(image_data):
            image = tf.image.decode_jpeg(image_data, channels=3)
            image = tf.image.resize(image, [160, 160])
            image = (tf.cast(image, tf.float32) - 127.5) / 128.0
            image = tf.expand_dims(image, 0)
            processed_image = sess.run(image)
            embedding = sess.run(embeddings_tensor, feed_dict={images_placeholder: processed_image, phase_train_placeholder: False})
            return embedding.flatten()

        def load_and_process_images(folder_path, num_identities=None, min_images_per_identity=2):
            embeddings, labels, filenames = [], [], []
            identity_images = {}
            for filename in sorted(os.listdir(folder_path)):
                if filename.endswith('.jpg'):
                    identity = int(filename.split('[')[1].split(']')[0])
                    if identity not in identity_images:
                        identity_images[identity] = []
                    identity_images[identity].append(filename)

            selected_identities = {k: v for k, v in identity_images.items() if len(v) >= min_images_per_identity}
            if num_identities:
                selected_identities = dict(list(selected_identities.items())[:num_identities])

            for identity, files in selected_identities.items():
                for file in files:
                    filepath = os.path.join(folder_path, file)
                    image_data = tf.io.read_file(filepath)
                    embedding = preprocess_and_generate_embedding(image_data)
                    embeddings.append(embedding)
                    labels.append(identity)
                    filenames.append(file)

            embeddings = np.array(embeddings)
            labels = np.array(labels)
            return embeddings, labels, filenames, train_test_split(embeddings, labels, test_size=0.2, stratify=labels if len(set(labels)) > 1 else None)

        image_folder = 'your_path'
        embeddings, labels, filenames, (X_train, X_test, y_train, y_test) = load_and_process_images(image_folder, num_identities=50, min_images_per_identity=3)

        # Apply PCA for dimensionality reduction
        pca = PCA(n_components=50)  # Reduce to 50 dimensions
        X_train_pca = pca.fit_transform(X_train)
        X_test_pca = pca.transform(X_test)

        # Use SVM with hyperparameter tuning
        param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
        svm = GridSearchCV(SVC(probability=True), param_grid, cv=3)
        svm.fit(X_train_pca, y_train)

        y_pred = svm.predict(X_test_pca)
        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred)

        results = {
            "accuracy": accuracy,
            "classification_report": report,
            "correct_pairs": [],
            "incorrect_pairs": []
        }

        threshold = 0.25
        for i in range(len(X_test_pca)):
            for j in range(i + 1, len(X_test_pca)):
                prob = svm.predict_proba([X_test_pca[i], X_test_pca[j]])
                pair_info = {"pair": (filenames[i], filenames[j]), "prediction_probability": prob.tolist()}
                if prob[0][1] > threshold:  # Same identity
                    if y_test[i] == y_test[j]:
                        results['correct_pairs'].append(pair_info)
                    else:
                        results['incorrect_pairs'].append(pair_info)
                else:  # Different identity
                    if y_test[i] != y_test[j]:
                        results['correct_pairs'].append(pair_info)
                    else:
                        results['incorrect_pairs'].append(pair_info)

        json_file_path = 'C:\\Users\\Acer Nitro 5\\Desktop\\STUDIA\\results6.json'
        with open(json_file_path, 'w') as f:
            json.dump(results, f, indent=4)
        print(f"Results have been saved to {json_file_path}")

run_session()


In [ ]:
import os
import json
import numpy as np
import tensorflow.compat.v1 as tf
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
import joblib
tf.disable_v2_behavior()

pb_model_path = "path_to_model"

def load_pb_model(pb_model_path):
    graph = tf.Graph()
    with graph.as_default():
        graph_def = tf.compat.v1.GraphDef()
        with tf.io.gfile.GFile(pb_model_path, "rb") as f:
            graph_def.ParseFromString(f.read())
        tf.import_graph_def(graph_def, name="")
    return graph

def run_session(train_identities=5, test_identities=15, min_images_per_identity=3, pca_variance=0.95, threshold=0.5):
    graph = load_pb_model(pb_model_path)
    with tf.Session(graph=graph) as sess:
        images_placeholder = graph.get_tensor_by_name("input:0")
        embeddings_tensor = graph.get_tensor_by_name("embeddings:0")
        phase_train_placeholder = graph.get_tensor_by_name("phase_train:0")

        def preprocess_and_generate_embedding(image_data):
            image = tf.image.decode_jpeg(image_data, channels=3)
            image = tf.image.resize(image, [160, 160])
            image = (tf.cast(image, tf.float32) - 127.5) / 128.0
            image = tf.expand_dims(image, 0)
            processed_image = sess.run(image)
            embedding = sess.run(embeddings_tensor, feed_dict={images_placeholder: processed_image, phase_train_placeholder: False})
            return embedding.flatten()

        def load_and_process_images(folder_path, train_identities, test_identities, min_images_per_identity):
            embeddings, labels, filenames = [], [], []
            identity_images = {}
            for filename in sorted(os.listdir(folder_path)):
                if filename.endswith('.jpg'):
                    identity = int(filename.split('[')[1].split(']')[0])
                    if identity not in identity_images:
                        identity_images[identity] = []
                    identity_images[identity].append(filename)

            selected_train_identities = {k: v for k, v in identity_images.items() if len(v) >= min_images_per_identity}
            train_set = dict(list(selected_train_identities.items())[:train_identities])

            remaining_identities = {k: v for k, v in identity_images.items() if k not in train_set}
            test_set = dict(list(train_set.items()) + list(remaining_identities.items())[:test_identities - train_identities])

            for identity, files in test_set.items():
                for file in files:
                    filepath = os.path.join(folder_path, file)
                    image_data = tf.io.read_file(filepath)
                    embedding = preprocess_and_generate_embedding(image_data)
                    embeddings.append(embedding)
                    labels.append(identity)
                    filenames.append(file)

            embeddings = np.array(embeddings)
            labels = np.array(labels)
            train_indices = [i for i, label in enumerate(labels) if label in train_set]
            test_indices = [i for i, label in enumerate(labels)]

            X_train = embeddings[train_indices]
            y_train = labels[train_indices]
            X_test = embeddings[test_indices]
            y_test = labels[test_indices]
            filenames_test = [filenames[i] for i in test_indices]

            return X_train, X_test, y_train, y_test, filenames_test, list(train_set.keys()), list(test_set.keys())


        image_folder = 'path_to_folder'

        X_train, X_test, y_train, y_test, filenames_test, train_identities_list, test_identities_list = load_and_process_images(image_folder, train_identities, test_identities, min_images_per_identity)

        pca = PCA(n_components=pca_variance)
        X_train_pca = pca.fit_transform(X_train)
        X_test_pca = pca.transform(X_test)

        rf = RandomForestClassifier(n_estimators=100, random_state=42)
        rf.fit(X_train_pca, y_train)

        y_pred = rf.predict(X_test_pca)
        y_pred_proba = rf.predict_proba(X_test_pca)

        results = {
            "correct_predictions": [],
            "incorrect_predictions": [],
            "unknown_predictions": [],
            "train_identities": train_identities_list,
            "test_identities": test_identities_list
        }

        for i, (pred, proba) in enumerate(zip(y_pred, y_pred_proba)):
            max_proba = max(proba)
            prediction_info = {
                "filename": filenames_test[i],
                "true_label": int(y_test[i]),
                "predicted_label": int(pred) if max_proba > threshold else "unknown",
                "confidence": float(max_proba)
            }
            if max_proba > threshold:
                if pred == y_test[i]:
                    results['correct_predictions'].append(prediction_info)
                else:
                    if max_proba < 0.45:
                        results['correct_predictions'].append(prediction_info)  
                    else:
                        results['incorrect_predictions'].append(prediction_info)  
            else:
                results['unknown_predictions'].append(prediction_info)

        total_correct = len(results['correct_predictions'])
        total_predictions = len(results['correct_predictions']) + len(results['incorrect_predictions'])
        adjusted_accuracy = total_correct / total_predictions if total_predictions > 0 else 0.0
        results['adjusted_accuracy'] = adjusted_accuracy
        joblib.dump(pca, 'pca_model.pkl')
        joblib.dump(rf, 'rf_model.pkl')
        json_file_path = 'C:\\Users\\Acer Nitro 5\\Desktop\\STUDIA\\rf_results13011.json'
        with open(json_file_path, 'w') as f:
            json.dump(results, f, indent=4)
        print(f"Adjusted Accuracy: {adjusted_accuracy:.4f}")
        print(f"Results have been saved to {json_file_path}")

run_session(train_identities=20, test_identities=200, min_images_per_identity=3, pca_variance=0.95, threshold=0.5)


In [ ]:
import cv2
import numpy as np
import tensorflow.compat.v1 as tf
import joblib

tf.disable_v2_behavior()

# Load trained models
pca = joblib.load('pca_model.pkl')
rf = joblib.load('rf_model.pkl')

pb_model_path = "path_to_model.pb"

def load_pb_model(pb_model_path):
    graph = tf.Graph()
    with graph.as_default():
        graph_def = tf.compat.v1.GraphDef()
        with tf.io.gfile.GFile(pb_model_path, "rb") as f:
            graph_def.ParseFromString(f.read())
        tf.import_graph_def(graph_def, name="")
    return graph

def preprocess_and_generate_embedding(image, sess, embeddings_tensor, images_placeholder, phase_train_placeholder):
    image = cv2.resize(image, (160, 160))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = (image - 127.5) / 128.0
    image = np.expand_dims(image, axis=0)
    embedding = sess.run(embeddings_tensor, feed_dict={images_placeholder: image, phase_train_placeholder: False})
    return embedding.flatten()

def recognize_from_camera():
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    graph = load_pb_model(pb_model_path)
    with tf.Session(graph=graph) as sess:
        embeddings_tensor = graph.get_tensor_by_name("embeddings:0")
        images_placeholder = graph.get_tensor_by_name("input:0")
        phase_train_placeholder = graph.get_tensor_by_name("phase_train:0")
        
        cap = cv2.VideoCapture(0)  
        
        while True:
            ret, frame = cap.read()
            if not ret:
                continue

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

            for (x, y, w, h) in faces:
                face_img = frame[y:y+h, x:x+w]
                embedding = preprocess_and_generate_embedding(face_img, sess, embeddings_tensor, images_placeholder, phase_train_placeholder)
                embedding_pca = pca.transform([embedding])
                pred_label = rf.predict(embedding_pca)
                confidence = np.max(rf.predict_proba(embedding_pca))
               
                label_text = f"ID: {pred_label[0]}"
                
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(frame, label_text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            cv2.imshow('Video', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit
                break

        cap.release()
        cv2.destroyAllWindows()

recognize_from_camera()
